# **Konkrete Fragen**

## Frage 1: Hängt die Wahrscheinlichkeit, dass bei einer Bestellung Trinkgeld gegeben wird, vom Trinkgeldverhalten bei früheren Bestellungen desselben Bestellers ab?

Hinweis: Im folgenden sprechen wir über Häufigkeiten, obwohl die Berechnungen auf relativen Häufigkeiten basieren. Da die Fallzahlen so hoch sind, kann der Unterschied vernachlässigt werden (Gesetz der großen Zahlen)

In [2]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt  


In [ ]:
df_aisles = pd.read_csv("data\\aisles.csv.zip")
df_departments = pd.read_csv("data\\departments.csv.zip")
df_order_products_s = pd.read_csv("data\\order_products_s.csv.zip")
df_orders_s = pd.read_csv("data\\orders_s.csv.zip")
df_products = pd.read_csv("data\\products.csv.zip")
df_tips = pd.read_csv("data\\tips.csv.zip")

In [4]:
# Zunächst werden die DataFrames 'Order' und 'Tip' zusammengeführt.
ord_tips = pd.merge(df_orders_s, df_tips, on='order_id')

In [5]:
# Anschließend werden die Spalten entfernt, die für diese Analyse nicht benötigt werden.
ord_tips = ord_tips.drop(columns=['order_dow', 'order_hour_of_day', 'order_id', 'days_since_prior_order', 'Unnamed: 0'])

In [6]:
# Die Spalte 'tip' wird von True/False in 1/0 umgewandelt, um die Berechnung relativer Häufigkeiten zu erleichtern.
ord_tips['tip'] = ord_tips['tip'].astype(int)

In [7]:
ord_tips.head()

,user_id,order_number,tip
0,1,1,0
1,1,2,0
2,1,3,0
3,1,4,0
4,1,5,0


In [8]:
# Da wir 'tip' der vorhergehenden und vorvorhergehenden Bestellungen benötigen, 
# definieren wir eine Funktion, die für beide Fälle verwendet werden kann.

def calculate_probability_of_tip(ord_tips, days_before):
    # DataFrame nach 'user_id' und 'order_number' sortieren
    ord_tips = ord_tips.sort_values(by=['user_id', 'order_number'])

    # Spalte 'tip' basierend auf der Anzahl der Tage zuvor verschieben
    ord_tips['previous_tip'] = ord_tips.groupby('user_id')['tip'].shift(days_before)

    # Bestellungen mit zu wenig Bestellhistorie entfernen
    ord_tips = ord_tips[ord_tips['order_number'] > days_before]

    # Relative Häufigkeit berechnen
    probability_df = ord_tips.groupby('previous_tip')['tip'].mean().round(2).reset_index()
    probability_df.columns = [f'previous_{days_before}_tip', 'für Trinkgeld']

    # Relative Häufigkeit für kein Trinkgeld berechnen
    probability_df['für kein Trinkgeld'] = 1 - probability_df['für Trinkgeld']

    return probability_df


In [83]:
def bar_probab_tip(vorhergehender_tag, n):
    # DataFrame umformen, um relative Häufigkeiten für 'Tip' und 'kein Tip' darzustellen
    df_stacked = vorhergehender_tag.melt( # melt verwendet, um geeignete Spalten zur Visualisierung zu haben 
        id_vars=f'previous_{n}_tip',
        value_vars=['für Trinkgeld', 'für kein Trinkgeld'],
        var_name='Wahrscheinlichkeit',
        value_name='Trinkgeld (Anteil)')
    
    df_stacked[f"previous_{n}_tip"] = ["Ja" if x == 1 else "Nein" for x in df_stacked[f"previous_{n}_tip"]]

    # Gestapeltes Balkendiagramm für relative Häufigkeiten erstellen
    fig = px.bar(
        df_stacked,
        x='Trinkgeld (Anteil)',
        y=f'previous_{n}_tip',
        color='Wahrscheinlichkeit',
        orientation='h', # Horizontale Ausrichtung der Balken.
        title=(
            f"Trinkgeldwahrscheinlichkeit in Abhängigkeit von dem vorherigen Trinkgeldverhalten<br>"
            f"<sup>Das aktuelle Trinkgeldverhalten wird von dem {'vor-' if n == 2 else ''}vorherigen Trinkgeldverhalten beeinflusst</sup>"),
        color_discrete_sequence=["#65437f","#c57c21"])
    
    # Layout anpassen: Achsentitel und gestapeltes Balkendiagramm.
    fig.update_layout(
        barmode='stack',
        xaxis_title='Trinkgeld (Anteil)',
        yaxis_title=f"Wurde bei der {'letzten' if n == 1 else 'vorletzen'} Bestellung Trinkgeld gegeben?",
        legend_title='Wahrscheinlichkeit')
    fig.show()


### a) Gibt es einen Zusammenhang bezüglich der vorhergehenden Bestellung?

In [10]:
#  Wir definieren eine neue Spalte 'previous_tip', die die vorherigen Trinkgelder für jeden Kunden enthält.
ord_tips['previous_tip'] = ord_tips.groupby('user_id')['tip'].shift(1)

In [11]:
# Die ersten Bestellungen der Kunden werden entfernt, da die Spalte 'previous_tip' für diese Bestellungen NaN-Werte enthält. 
ord_tips_vor = ord_tips.dropna(subset=['previous_tip'])

In [12]:
# Erstellen einer Kreuztabelle, um die Verteilung der Trinkgelder ('tip') im Verhältnis zu den vorherigen Trinkgeldern ('previous_tip') zu analysieren.
pd.crosstab(ord_tips_vor.tip, ord_tips_vor.previous_tip,normalize='columns')

previous_tip,0.0,1.0
tip,,
0,0.776135,0.284972
1,0.223865,0.715028


In [13]:
# Wir betrachten die absoluten Zahlen, um sicherzustellen, dass die Trinkgeldraten überhaupt noch aussagekräftig sind. 
pd.crosstab(ord_tips_vor.tip, ord_tips_vor.previous_tip)

previous_tip,0.0,1.0
tip,,
0,679280,197972
1,195929,496736


Die Fallzahlen sind ausreichend.

In [84]:
# Visualisierung
vorhergehende_bestellung = calculate_probability_of_tip(ord_tips, 1)
bar_probab_tip(vorhergehende_bestellung, 1)

Wenn ein Kunde bei seiner letzten Bestellung kein Trinkgeld gegeben hat, liegt die Wahrscheinlichkeit, dass er bei der nächsten Bestellung Trinkgeld gibt, bei 22 %.    
Hat der Kunde hingegen bei der vorherigen Bestellung Trinkgeld gegeben, steigt die Wahrscheinlichkeit, dass er erneut Trinkgeld gibt auf 72 %.

-> Das bedeutet, dass ein Kunde mehr als dreimal wahrscheinlicher erneut Trinkgeld gibt, wenn er bereits bei der letzten Bestellung Trinkgeld gegeben hat, im Vergleich dazu, wenn er keines gegeben hat.

Ähnlich verhält es sich, wenn ein Kunde bei seiner letzten Bestellung kein Trinkgeld gegeben hat: Die Wahrscheinlichkeit, dass er auch bei der nächsten Bestellung keines gibt, liegt bei 78 %.

Zusammenfassend lässt sich sagen, dass das Trinkgeldverhalten eines Kunden stark von seinem Verhalten bei der vorherigen Bestellung abhängt. Kunden, die bereits Trinkgeld gegeben haben, neigen dazu, dies wieder zu tun, während Kunden, die kein Trinkgeld gegeben haben, eher dabei bleiben, keines zu geben.

### b) Gibt es einen Zusammenhang bezüglich der vor-vorhergehenden Bestellung?

In [15]:
# Spalte 'preprevious_tip' erstellen, die das Trinkgeld aus der vorvorherigen Bestellung enthält.
ord_tips['preprevious_tip'] = ord_tips.groupby('user_id')['tip'].shift(2)

In [16]:
# Die ersten zwei Bestellungen jedes Kunden ausschließen.
ord_tips_vorvor = ord_tips.dropna(subset=['preprevious_tip'])

In [17]:
# Kreuztabelle erstellen, um die relative Häufigkeit der aktuellen Trinkgeldentscheidung ('tip') 
# im Verhältnis zur vorvorherigen Trinkgeldentscheidung ('preprevious_tip') zu analysieren. 
pd.crosstab(ord_tips_vorvor.tip, ord_tips_vorvor.preprevious_tip, normalize='columns')

preprevious_tip,0.0,1.0
tip,,
0,0.742474,0.319731
1,0.257526,0.680269


In [85]:
vorvorhergehende_bes = calculate_probability_of_tip(ord_tips, 2)
bar_probab_tip(vorvorhergehende_bes, 2)

Wenn ein Kunde bei seiner vorletzten Bestellung kein Trinkgeld gegeben hat, liegt die Wahrscheinlichkeit, dass er bei der nächsten Bestellung Trinkgeld gibt, bei 26 %. Hat der Kunde jedoch bei der vorletzten Bestellung Trinkgeld gegeben, steigt die Wahrscheinlichkeit, dass er erneut Trinkgeld gibt, auf 68 %.

Das bedeutet, dass Kunden eher Trinkgeld geben, wenn sie dies bereits bei ihrer vorletzten Bestellung getan haben. Allerdings ist der Einfluss des Trinkgeldverhaltens bei der vorletzten Bestellung weniger stark ausgeprägt als der Einfluss des Verhaltens bei der letzten Bestellung.

In der Analyse zur vor-vorletzten Bestellung mussten die ersten zwei Bestellungen eines Kunden ausgeschlossen werden, da es zu diesen keine vor-vorletzten Bestellungen gibt. Bei der Analyse zur Aufgabe a) zur letzten Bestellung wurde aber nur die erste Bestellung eines Kunden ausgeschlossen. Um unsere Ergebnisse vergleichen zu können, untersuchen wir, ob es einen signifikaten unterschied macht, wenn wir in der ersten Analyse auch die zweite Bestellung ausschließen.

In [19]:
# Ohne zweite Bestellung
pd.crosstab(ord_tips_vorvor.tip, ord_tips_vorvor.previous_tip,normalize='columns')

previous_tip,0.0,1.0
tip,,
0,0.772367,0.281732
1,0.227633,0.718268


In [20]:
# Mit zweiter Bestellung
pd.crosstab(ord_tips_vor.tip, ord_tips_vor.previous_tip,normalize='columns')

previous_tip,0.0,1.0
tip,,
0,0.776135,0.284972
1,0.223865,0.715028


Aus den beiden Tabellen lässt sich erkennen, dass das Ausschließen der zweiten Bestellungen nur einen sehr geringen Einfluss hat, der vernachlässigt werden kann.   
-> Also können wir unsere Ergebnisse aus den vorherigen Analysen vergleichen.

Es scheint, dass das Trinkgeldverhalten bei der vorletzten Bestellung zwar die Wahrscheinlichkeit beeinflusst, bei der nächsten Bestellung Trinkgeld zu geben, dieser Einfluss jedoch weniger bedeutend ist als das Trinkgeldverhalten bei der vorherigen Bestellung.

### c) Liefert das Trinkgeldverhalten der vor-vorhergehende Bestellung Informationen auch über das hinaus, was bereits aus der vorhergehenden Bestellung abgelesen werden kann?

In [21]:
# Gruppieren nach 'tip', 'preprevious_tip' und 'previous_tip', um die Häufigkeit jeder Kombination zu zählen
tips_both = ord_tips_vorvor.groupby(['tip', 'preprevious_tip', 'previous_tip']).size().reset_index(name='count')

# Normalisieren der Häufigkeiten innerhalb jeder Kombination von 'preprevious_tip' und 'previous_tip'
tips_both['relative_frequency'] = tips_both['count'] / tips_both.groupby(['preprevious_tip', 'previous_tip'])['count'].transform('sum')

In [22]:
tips_both.sort_values(by="preprevious_tip", inplace=True, ascending=False)

In dieser Analyse geht es um die Frage, ob ein direkter Zusammenhang zwischen dem Trinkgeld und dem Trinkgeldverhalten der vor-vorhergehenden Bestellung besteht, selbst wenn der Einfluss des Trinkgeldverhaltens der vorhergehenden Bestellung bereits berücksichtigt wurde.

Wenn kein direkter Zusammenhang zwischen Trinkgeld von der aktuellen und der vor-vorherigen Bestellung besteht, sollte sich die Wahrscheinlichkeit für Trinkgeld der aktuellen Bestellung ausschließlich durch das Trinkgeld der vorherigen Bestellung erklären lassen.

Um dies zu überprüfen, betrachten wir die bedingten Wahrscheinlichkeiten:

In [24]:
pd.crosstab(ord_tips_vorvor.tip, [ord_tips_vorvor.preprevious_tip,ord_tips_vorvor.previous_tip],normalize='columns')

preprevious_tip       0.0                 1.0          
previous_tip          0.0       1.0       0.0       1.0
tip                                                    
0                0.825825  0.461901  0.594185  0.211156
1                0.174175  0.538099  0.405815  0.788844

In [81]:
tips_both_viz = tips_both.copy()
tips_both_viz["tip"] = ["Ja" if x == 1 else "Nein" for x in tips_both_viz["tip"]]
tips_both_viz["previous_tip"] = ["Ja" if x == 1 else "Nein" for x in tips_both_viz["previous_tip"]]
tips_both_viz["preprevious_tip"] = ["Ja" if x == 1 else "Nein" for x in tips_both_viz["preprevious_tip"]]
fig = px.bar(tips_both_viz[tips_both_viz["previous_tip"]=="Ja"], x="preprevious_tip", y="relative_frequency", facet_row="tip",
       labels={"previous_tip":"Wurde in der vorherigen Bestellung Trinkgeld gegeben?",
               "preprevious_tip":"Wurde in der vorvorherigen Bestellung Trinkgeld gegeben?",
               "tip":"Trinkgeld?",
               "relative_frequency":"Anteil"},
               height=700,
               color="preprevious_tip",
               title="Einfluss der vorvorherigen Bestellung auf Bestellungen bei denen bei der vorherigen Bestellung Trinkgeld gegeben wurde<br><sub>Das Trinkgeldverhalten aus der vorvorherigen Bestellung liefert extra Informationen",
color_discrete_sequence=["#65437f","#c57c21"] ,
range_y=[0,1])

fig.update_traces(showlegend = False)
fig.show()

In [82]:
fig = px.bar(tips_both_viz[tips_both_viz["previous_tip"]=="Nein"], x="preprevious_tip", y="relative_frequency", facet_row="tip",
       labels={"previous_tip":"Wurde in der vorherigen Bestellung Trinkgeld gegeben?",
               "preprevious_tip":"Wurde in der vorvorherigen Bestellung Trinkgeld gegeben?",
               "tip":"Trinkgeld?",
               "relative_frequency":"Anteil"},
               height=700,
               color="preprevious_tip",
               title="Einfluss der vorvorherigen Bestellung auf Bestellungen bei denen bei der vorherigen Bestellung kein Trinkgeld gegeben wurde<br><sub>Das Trinkgeldverhalten aus der vorvorherigen Bestellung liefert wieder extra Informationen",
               color_discrete_sequence=["#65437f","#c57c21"],
               range_y=[0,1])

fig.update_traces(showlegend = False)
fig.show()

Das bedeutet, dass ein direkter Zusammenhang zwischen Trinkgeld und vorletzten Trinkgeldverhalten besteht, selbst wenn das letzte Trinkgeldverhalten bereits berücksichtigt wurde. Mit anderen Worten: Das Trinkgeldverhalten der vor-vorhergehenden Bestellung liefert zusätzlich Information über das Trinkgeldverhalten hinaus, was bereits durch das Trinkgeldverhalten der vorhergehenden Bestellung erklärt werden kann.

# Frage 2

### a) Gibt es einen Zusammenhang zwischen dem Trinkgeldverhalten und den Departments, aus denen bestellt wird?  

In [27]:
# Zunächst werden die DataFrames 'order_products', 'products', 'departments', 'tips' zusammengeführt.
tips_order_products = df_order_products_s.merge(df_products, on= 'product_id', how= 'inner' )
tips_order_products = tips_order_products.merge(df_departments, on= 'department_id', how= 'inner' )
tips_order_products = tips_order_products.merge(df_tips, on= 'order_id', how= 'inner' )

In [28]:
# Anschließend werden die Spalten entfernt, die für diese Analyse nicht benötigt werden.
tips_order_products = tips_order_products.drop(columns= ['add_to_cart_order', 'aisle_id', 'Unnamed: 0'])
# Die Spalte 'tip' wird von True/False in 1/0 umgewandelt, um die Berechnung relativer Häufigkeiten zu erleichtern.
tips_order_products['tip'] = tips_order_products['tip'].astype(int)

In [29]:
tips_order_products.head()

,order_id,product_id,product_name,department_id,department,tip
0,2,33120,Organic Egg Whites,16,dairy eggs,0
1,2,28985,Michigan Organic Kale,4,produce,0
2,2,9327,Garlic Powder,13,pantry,0
3,2,45918,Coconut Butter,13,pantry,0
4,2,30035,Natural Sweetener,13,pantry,0


In [30]:
order_dep_uni = tips_order_products[['order_id', 'department_id','department', 'tip']] 

# Ein Department kann in einer Bestellung mehrfach vorkommen.
order_dep_uni = order_dep_uni.drop_duplicates()
# Doppelte Department-Einträge für jede Bestellung entfernen.

In [31]:
order_dep_uni.head()

,order_id,department_id,department,tip
0,2,16,dairy eggs,0
1,2,4,produce,0
2,2,13,pantry,0
9,4,3,bakery,1
10,4,11,personal care,1


In [32]:
# Daten nach Department gruppieren und Durchschnitt der Trinkgelder berechnen
tip_by_department = order_dep_uni.groupby('department')['tip'].mean().reset_index().rename(columns={'tip': 'tip_percentage'}).round(4).sort_values(by='tip_percentage', ascending=False)
tip_by_department.head(10)

,department,tip_percentage
0,alcohol,0.8057
5,bulk,0.5532
7,dairy eggs,0.4903
1,babies,0.4897
4,breakfast,0.4760
19,produce,0.4559
13,meat seafood,0.4498
15,other,0.4419
17,personal care,0.4405
11,household,0.4340


In [33]:
# Anteil ohne Trinkgeld berechnen.
tip_by_department['no_tip_percentage'] = 1- tip_by_department['tip_percentage']
tip_by_department.head()

,department,tip_percentage,no_tip_percentage
0,alcohol,0.8057,0.1943
5,bulk,0.5532,0.4468
7,dairy eggs,0.4903,0.5097
1,babies,0.4897,0.5103
4,breakfast,0.4760,0.5240


In [75]:
# Departments mit einem Trinkgeldanteil von mehr als 0% filtern.
filtered_data = tip_by_department[tip_by_department['tip_percentage'] > 0]

# Trinkgeldanteil in Prozent umrechnen und auf zwei Nachkommastellen runden.
filtered_data["tip_percentage"] = (filtered_data["tip_percentage"] * 100).round(2)

top_department = filtered_data.iloc[0]  # Department mit dem höchsten Trinkgeldanteil

subtitle = f"Bestellungen mit Produkten aus dem Department '{top_department['department']}' haben den höchsten Trinkgeldanteil "

# Balkendiagramm erstellen, das den Trinkgeldanteil pro Department zeigt, mit Hervorhebung des Top-Departments.
fig = px.bar(
    filtered_data, 
    x='department', 
    y='tip_percentage', 
    title='Trinkgeldverhalten nach Departments',
    labels={'department': 'Departments', 'tip_percentage': 'Trinkgeld in (%)'}, 
    text='tip_percentage',
    color="department",
    color_discrete_sequence=["#65437f"] + ["lightgrey" for _ in range(len(filtered_data["department"]) - 1)],
    width=1500
    
)

# Texte außerhalb der Balken positionieren
fig.update_traces(textposition='outside', showlegend=False)
fig.update_layout(
    margin=dict(t=68),  
    title={'text': f"Trinkgeldverhalten nach Departments<br><sup>{subtitle}</sup>"}
)

# Bereich der y-Achse auf 0 bis 100 festlegen.
fig.update_yaxes(range=[0, 100])

fig.show()


### b) Gibt es einzelne Produkte, die die Trinkgeldwahrscheinlichkeit besonders stark beeinflussen? (Ermitteln Sie die TOP10 und FLOP10)

#### Berechnung Trinkgeldanteil pro Produkt

In [35]:
# Produkte nach ID und Name gruppieren, die Anzahl der Produkte und den durchschnittlichen Trinkgeldanteil berechnen.
tip_produkt = tips_order_products.groupby(['product_id', 'product_name']).agg(
    product_count=('product_id', 'count'),
    tip_proportion=('tip', 'mean'))
tip_produkt.head()

,,product_count,tip_proportion
product_id,product_name,,
1,Chocolate Sandwich Cookies,1021,0.337904
2,All-Seasons Salt,33,0.363636
3,Robust Golden Unsweetened Oolong Tea,130,0.592308
4,Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce,164,0.219512
5,Green Chile Anytime Sauce,4,0.500000


In [36]:
tip_produkt.describe()

,product_count,tip_proportion
count,49258.000000,49258.000000
mean,343.948922,0.388080
std,2507.424673,0.191169
min,1.000000,0.000000
25%,9.000000,0.286829
50%,32.000000,0.384615
75%,138.000000,0.481978
max,244850.000000,1.000000


#### Verwerfen der unteren 50% der Produkte, da diese nicht oft genug gekauft wurden, um statistisch relevante Aussagen zu machen

In [38]:
tip_produkt = tip_produkt[tip_produkt.product_count >=32]

#### Top 10 Produkte

In [39]:
product_top = tip_produkt.sort_values(by=['tip_proportion', 'product_count'], ascending=[False, False])
product_top.head(10)

,,product_count,tip_proportion
product_id,product_name,,
20341,Fine Champagne Cognac,39,0.974359
22283,"Zinfandel, Lodi, California",36,0.972222
33712,Organic Wheatgrass Sprouts,34,0.970588
25143,Curieux Ale,33,0.969697
27857,Dos Equis Amber Lager,33,0.969697
45328,"Flavored Vodka, Peach",54,0.962963
1916,Merlot Wine,99,0.959596
11878,"Cabernet Sauvignon, Paso Robles, 2012",58,0.948276
44150,Non Alcoholic Beer,77,0.948052


#### Flop 10 Produkte

In [40]:
product_flop = tip_produkt.sort_values(by=['tip_proportion', 'product_count'], ascending=[True, False])
product_flop.head(10)

,,product_count,tip_proportion
product_id,product_name,,
45228,Temptations Turkey Flavor Treats,39,0.000000
39210,Super Premium Chocolate Ice Cream,64,0.015625
11951,Singles Deep Dish Meat Trio Pizzas,40,0.025000
15559,Cocoa & Vanilla Gluten Free Bunny Cookies,34,0.029412
43411,Iced Lemon Cake Slice,33,0.030303
47889,Crispy Chocolate Chip Cookies,32,0.031250
24538,"Frozen Dessert, Non-Dairy, Mocha Pie",52,0.038462
45523,Dark 83% Pure Dark Chocolate Bite,80,0.050000
36411,Sourdough Cottage Bread,40,0.050000


### c) Besteht der Zusammenhang aus (b) zusätzlich zu dem aus (a), d.h. beeinflusst das Vorhandensein der Produkte aus (b) die Trinkgeldwahrscheinlichkeit auch über das hinaus, was durch das Vorhandensein der entsprechenden Departments schon absehbar ist?

In [41]:
tips_order_product = tips_order_products[tips_order_products["product_id"].isin(tip_produkt.reset_index()["product_id"])]

In [42]:
# Kreuztabelle erstellen, um den Trinkgeldanteil pro Produkt und Department zu berechnen, normalisiert nach Zeile.
product_dep_tip_precentage = pd.crosstab([tips_order_product.product_name,tips_order_product.department],tips_order_product.tip, normalize="index")

product_dep_tip_precentage.reset_index(inplace=True)

# Spalten umbenennen, um den Anteil mit und ohne Trinkgeld zu kennzeichnen.
product_dep_tip_precentage = product_dep_tip_precentage.rename(columns={0:"product_no_tip_percentage", 1:"product_tip_percentage"})
tip_by_department_renamed = tip_by_department.rename(columns={"tip_percentage":"department_tip_percentage", "no_tip_percentage":"department_no_tip_percentage"})

# Produkt- und Department-Daten mit den aggregierten Department-Trinkgeldanteilen basierend auf dem Department verknüpfen.
merged_prod_dep_only_dep_tips = product_dep_tip_precentage.merge(tip_by_department_renamed, on="department")


In [43]:
# Differenz zwischen Produkt-Trinkgeldanteil und Department-Trinkgeldanteil berechnen (absoluter Wert).
merged_prod_dep_only_dep_tips["tip_difference_from_department"] = abs((merged_prod_dep_only_dep_tips["product_tip_percentage"] - merged_prod_dep_only_dep_tips["department_tip_percentage"]))

# Gibt an, ob die Trinkgeldwahrscheinlichkeit zusätzlich zum Department auch vom Produkt beeinflusst wird
merged_prod_dep_only_dep_tips["Product Probability Independent?"] = merged_prod_dep_only_dep_tips["tip_difference_from_department"] < 0.05

# Prozentsatz der Produkte berechnen, die die Trinkgeldwahrscheinlichkeit zusätzlich zum Department beeinflussen.
independent_percentage = (1 - merged_prod_dep_only_dep_tips["Product Probability Independent?"].sum() / merged_prod_dep_only_dep_tips["product_name"].nunique())*100

print(f"Ungefähr {independent_percentage:.2f}% der Produkte beeinflussen die Trinkgeldwahrscheinlichkeit über das, was wird bereits durch das Vorhandensein des Departments in einer Bestellung wissen, hinaus.")

Ungefähr 52.50% der Produkte beeinflussen die Trinkgeldwahrscheinlichkeit über das, was wird bereits durch das Vorhandensein des Departments in einer Bestellung wissen, hinaus.


In [ ]:
# Daten nach der Differenz der Trinkgeldwahrscheinlichkeit sortieren und den Index zurücksetzen.
df_viz = merged_prod_dep_only_dep_tips.sort_values(by="tip_difference_from_department", ascending=True).reset_index()

# Differenz der Trinkgeldwahrscheinlichkeit in Prozent umrechnen.
df_viz["tip_difference_from_department"] = df_viz["tip_difference_from_department"] * 100

# Werte in der Spalte "Product Probability Independent?" in "Ja" oder "Nein" umwandeln.
df_viz["Product Probability Independent?"] = ["Nein" if x == True else "Ja" for x in df_viz["Product Probability Independent?"]]

# Liniendiagramm erstellen, das den Unterschied in der Trinkgeldwahrscheinlichkeit pro Produkt zeigt, farbcodiert nach der Relevanz des Produkts.
fig = px.line(df_viz, x=df_viz.index,  y="tip_difference_from_department", 
    color="Product Probability Independent?",
    title=(
        f"Anteil der Produkte, die uns signifikante Informationen über das Trinkgeldverhalten liefern<br>"
        f"<sub>Ca. {independent_percentage:.2f}% der Produkte beeinflussen die Trinkgeldwahrscheinlichkeit über das, "
        "was wir bereits durch das Vorhandensein des Departments in einer Bestellung wissen, hinaus.</sub>"),
    labels={
        "_index": "Produkte", 
        "tip_difference_from_department": "Unterschied Trinkgeldwahrscheinlichkeit: Produkt vs. Department (%)",
        "Product Probability Independent?": "Produkt liefert Zusatzinformationen"},
    height=600,
    width=1500,
    color_discrete_sequence=["grey", "#fe001c"],)

# Horizontale Linie bei 5% hinzufügen, um den Schwellenwert für signifikante Unterschiede darzustellen.
fig.add_hline(y=5, annotation_text="Cut-Off")

fig.show()



#### Einfluss der Top/Flop 10 Produkte auf die Trinkgeldwahrscheinlichkeit

In [45]:
# Filter anwenden, um nur die Produkte anzuzeigen, die unter den Top 10 Produkten sind.
merged_prod_dep_only_dep_tips[merged_prod_dep_only_dep_tips["product_name"].isin(product_top.head(10).reset_index()["product_name"])]

,product_name,department,product_no_tip_percentage,product_tip_percentage,department_tip_percentage,department_no_tip_percentage,tip_difference_from_department,Product Probability Independent?
2895,"Cabernet Sauvignon, Paso Robles, 2012",alcohol,0.051724,0.948276,0.8057,0.1943,0.142576,False
5683,Curieux Ale,alcohol,0.030303,0.969697,0.8057,0.1943,0.163997,False
6365,Dos Equis Amber Lager,alcohol,0.030303,0.969697,0.8057,0.1943,0.163997,False
7188,Fine Champagne Cognac,alcohol,0.025641,0.974359,0.8057,0.1943,0.168659,False
7298,"Flavored Vodka, Peach",alcohol,0.037037,0.962963,0.8057,0.1943,0.157263,False
12126,Merlot Wine,alcohol,0.040404,0.959596,0.8057,0.1943,0.153896,False
13279,Non Alcoholic Beer,alcohol,0.051948,0.948052,0.8057,0.1943,0.142352,False
16332,Organic Wheatgrass Sprouts,produce,0.029412,0.970588,0.4559,0.5441,0.514688,False
23883,Vodka 80 Proof,alcohol,0.053571,0.946429,0.8057,0.1943,0.140729,False
24710,"Zinfandel, Lodi, California",alcohol,0.027778,0.972222,0.8057,0.1943,0.166522,False


&rarr; Die Top 10 Produke haben einen positiven Einfluss auf die Trinkgeldwahrscheinlichkeit. Interessant ist der sehr starke Unterschied bei "Organic Wheatgrass Sprouts" von über +50% Unterschied

In [46]:
# Filter anwenden, um nur die Produkte anzuzeigen, die unter den Flop 10 Produkten (basierend auf einer vorher definierten Metrik) sind.
merged_prod_dep_only_dep_tips[merged_prod_dep_only_dep_tips["product_name"].isin(product_flop.head(10).reset_index()["product_name"])]

,product_name,department,product_no_tip_percentage,product_tip_percentage,department_tip_percentage,department_no_tip_percentage,tip_difference_from_department,Product Probability Independent?
4761,Cocoa & Vanilla Gluten Free Bunny Cookies,snacks,0.970588,0.029412,0.4157,0.5843,0.386288,False
5520,Crispy Chocolate Chip Cookies,snacks,0.968750,0.031250,0.4157,0.5843,0.384450,False
5851,Dark 83% Pure Dark Chocolate Bite,snacks,0.950000,0.050000,0.4157,0.5843,0.365700,False
7791,"Frozen Dessert, Non-Dairy, Mocha Pie",frozen,0.961538,0.038462,0.3585,0.6415,0.320038,False
8523,Gluten Free Sub Sandwich Roll,bakery,0.948718,0.051282,0.3902,0.6098,0.338918,False
10101,Iced Lemon Cake Slice,bakery,0.969697,0.030303,0.3902,0.6098,0.359897,False
20348,Singles Deep Dish Meat Trio Pizzas,frozen,0.975000,0.025000,0.3585,0.6415,0.333500,False
20870,Sourdough Cottage Bread,bakery,0.950000,0.050000,0.3902,0.6098,0.340200,False
21780,Super Premium Chocolate Ice Cream,frozen,0.984375,0.015625,0.3585,0.6415,0.342875,False
22223,Temptations Turkey Flavor Treats,pets,1.000000,0.000000,0.3737,0.6263,0.373700,False


&rarr; Die Flop 10 Produkte haben einen starken negativen Einfluss auf die Trinkgeldwahrscheinlichkeit

## Frage 3: Gibt es einen Zusammenhang zwischen dem Trinkgeldverhalten und der Tageszeit, dem Wochentag, oder dem Zeitabstand zur vorhergehenden Bestellung? 

In [47]:
# Zunächst werden die Tabellen "orders" und "tips" miteinander verknüpft.
df = pd.merge(df_orders_s, df_tips, on='order_id', how='left')

df.drop(columns="Unnamed: 0", inplace=True)

# Die Spalte "tip" wird von True/False in 1/0 umgewandelt.
df['tip'] = df['tip'].astype(int)

df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip
0,2539329,1,1,2,8,NaN,0
1,2398795,1,2,3,7,15.0,0
2,473747,1,3,3,12,21.0,0
3,2254736,1,4,4,7,29.0,0
4,431534,1,5,4,15,28.0,0


In [48]:
# Kreuztabelle erstellen, um die Anzahl der Bestellungen mit und ohne Trinkgeld nach der Uhrzeit zu vergleichen.
tip_hod_comparison = pd.crosstab(df["order_hour_of_day"], df["tip"])
tip_hod_comparison

tip,0,1
order_hour_of_day,,
0,5221,6003
1,2967,3244
2,1842,1931
3,1365,1397
4,1342,1406
5,2759,2005
6,8542,6462
7,25244,19897
8,48500,39013


&rarr; Signifikanztest nicht nötig, da für jede Tageszeit mindestens 2500 Bestellungen existieren

### Gibt es einen Zusammenhang zwischen dem Trinkgeldverhalten und der Tageszeit?

In [76]:
tip_hod_comparison_viz = pd.crosstab(df["order_hour_of_day"], df["tip"], normalize="index").reset_index()
fig = px.bar(
    tip_hod_comparison_viz,
    x='order_hour_of_day',
    y=1, 
    title="Anteil des Trinkgelds nach Tageszeit<br><sub>Von 5 bis 18 Uhr ist der Anteil an Trinkgeld geringer und steigt am Abend an",
    labels={'order_hour_of_day': 'Tageszeit (Stunde)', 'tip_percentage': 'Trinkgeldanteil'},
    height=500, # Höhe des Diagramms
    color_discrete_sequence=["#65437f"]
)
# Y-Achse auf den Bereich 0 bis 1 begrenzen, da es um Prozentwerte geht
fig.update_layout(yaxis=dict(range=[0,1]))
fig.show()

###  Gibt es einen Zusammenhang zwischen dem Trinkgeldverhalten und dem Wochentag?

In [50]:
# Kreuztabelle erstellen, um die Anzahl der Bestellungen mit und ohne Trinkgeld nach dem Wochentag zu vergleichen.
tips_dow_comparison = pd.crosstab(df["order_dow"], df["tip"])
tips_dow_comparison

tip,0,1
order_dow,,
0,138307,154438
1,133199,154377
2,141113,87705
3,132305,81110
4,127579,81036
5,134991,87211
6,135414,84236


&rarr; Es gibt genug Bestellungen pro Wochentag, dass die Ergebnisse genügend Aussagekraft haben

In [79]:
# Kreuztabelle erstellen, um den Trinkgeldanteil pro Wochentag zu berechnen
tips_dow_comparison_viz = pd.crosstab(df["order_dow"], df["tip"], normalize="index").reset_index()

# Balkendiagramm erstellen, das den Anteil der Bestellungen mit Trinkgeld pro Wochentag zeigt.
fig = px.bar(tips_dow_comparison_viz, x="order_dow", y=True,
        title="Anteil der Bestellungen pro Wochentag, an dem es Trinkgeld gab<br><sub>An den Wochentagen 0 und 1 gab es anteilig bei mehr Bestellungen Trinkgeld als an den restlichen Tagen.",
        labels={"True":"Trinkgeldanteil (%)",
                "order_dow":"Wochentag"},
        height= 600,    color_discrete_sequence=["#65437f"],
        width = 1500)

# Bereich der y-Achse auf 0 bis 1 setzen.
fig.update_layout(yaxis=dict(range=[0,1]))
fig.show()

&rarr; Anscheinend gibt es einen Zusammenhang zwischen Wochentag und Trinkgeldwahrscheinlichkeit

#### Lässt sich dieser Zusammenhang durch die Trinkgeldwahrscheinlichkeit der Departments erklären?

In [52]:
tips_orders = tips_order_products.merge(df_orders_s, on= 'order_id', how= 'inner' )

In [53]:
tips_orders = tips_orders.drop(columns= ['product_id', 'product_name']) 

In [54]:
tips_orders.drop_duplicates()

,order_id,department_id,department,tip,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,16,dairy eggs,0,202279,3,5,9,8.0
1,2,4,produce,0,202279,3,5,9,8.0
2,2,13,pantry,0,202279,3,5,9,8.0
9,4,3,bakery,1,178520,36,1,9,7.0
10,4,11,personal care,1,178520,36,1,9,7.0
...,...,...,...,...,...,...,...,...,...
16942229,3421058,12,meat seafood,1,136952,20,3,18,15.0
16942230,3421058,1,frozen,1,136952,20,3,18,15.0
16942231,3421058,7,beverages,1,136952,20,3,18,15.0
16942232,3421058,19,snacks,1,136952,20,3,18,15.0


In [55]:
# Gruppieren nach Department und Wochentag, dann Berechnung des Trinkgeldanteil
department_tip_percentages = (tips_orders.groupby(["department", "order_dow"])["tip"].mean() .reset_index())

# Umbenennen der Spalte für bessere Lesbarkeit
department_tip_percentages.rename(columns={"tip": "tip_percentage"}, inplace=True)

fig = px.line(
    department_tip_percentages,
    x="order_dow",  # Wochentag auf der x-Achse
    y="tip_percentage",  # Trinkgeldanteil auf der y-Achse
    color="department",  # Linienfarben nach Department
    markers=True,  # Markierungen für Datenpunkte hinzufügen
    labels={"tip_percentage": "Trinkgeldanteil", "order_dow": "Wochentag"},  # Achsenbeschriftungen
    title="Trinkgeldanteil nach Department und Wochentag<br><sub>An den Tagen 0 und 1 haben fast alle Departments einen höheren Trinkgeldanteil im Vergleich zum Rest der Woche.")  # Diagrammtitel

# Formatierung der y-Achse als Prozentwerte
fig.update_layout(yaxis=dict(tickformat=".0%", range=[0,1]),xaxis=dict(range=[0,6]))

# Diagramm anzeigen
fig.show()

&rarr; Nein, der Zusammenhang von Wochentag und Trinkgeldwahrscheinlichkeit bleibt auch unter der Berücksichtigung der Departments bestehen.

### Trinkgeldverhalten nach Wochentag und Tageszeit

In [87]:
# Durchschnittlicher Trinkgeldanteil je Wochentag und Stunde berechnen.
tip_by_dow_hour = df.groupby(['order_dow', 'order_hour_of_day'])['tip'].mean().reset_index()

# Heatmap erstellen, die den Trinkgeldanteil nach Tageszeit und Wochentag visualisiert.
fig = px.imshow(
    tip_by_dow_hour.pivot(index='order_hour_of_day', columns='order_dow', values='tip'),
    labels=dict(x='Wochentag', y='Tageszeit', color='Prozentualer Anteil'),
    title=(
        'Trinkgeldverhalten nach Tageszeit und Wochentag<br>'
        '<sub>An Wochentag 0 und 1 wird am meisten Trinkgeld gegeben und tagsüber wird an allen Wochentagen '
        'weniger Trinkgeld gegeben als abends und nachts.</sub>'
    ),
    template="plotly_white",
    height=600,
    width=1500,
    aspect='auto',)

fig.update_yaxes(autorange=False, range=[0, 23])

fig.show()


### Gibt es einen Zusammenhang zwischen dem Trinkgeldverhalten und dem Zeitabstand zur vorhergehenden Bestellung?

In [80]:
# Kreuztabelle erstellen, um den Trinkgeldanteil basierend auf den Tagen seit der letzten Bestellung zu berechnen.
tips_dspo_comparison_viz = pd.crosstab(df["days_since_prior_order"], df["tip"], normalize="index").reset_index()

# Liniendiagramm erstellen, das den Trinkgeldanteil in Abhängigkeit von der Zeit seit der letzten Bestellung zeigt,
px.line(tips_dspo_comparison_viz, x="days_since_prior_order", y=True,
        title="Anteil an Bestellungen bei denen es Trinkgeld gab, in Abhängigkeit von dem Abstand zur letzten Bestellung<br><sub>Am häufigsten gibt es Trinkgeld bei Bestellungen, bei denen die letzte Bestellung eine Woche her ist. Danach sinkt die Trinkgeldwahrscheinlichkeit mit zunehmender Zeit die zwischen Bestellungen vergeht.",
        labels={"True":"Prozent der Bestellungen (%)", "days_since_prior_order": "Tage seit der letzten Bestellung"},    color_discrete_sequence=["#65437f"], height=600, width=1500)